In [30]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas_profiling

In [31]:
df = pd.read_csv("5G_energy_consumption_dataset.csv")

In [32]:
df.head()

,Time,BS,Energy,load,ESMODE,TXpower
0,20230101 010000,B_0,64.275037,0.487936,0.0,7.101719
1,20230101 020000,B_0,55.904335,0.344468,0.0,7.101719
2,20230101 030000,B_0,57.698057,0.193766,0.0,7.101719
3,20230101 040000,B_0,55.156951,0.222383,0.0,7.101719
4,20230101 050000,B_0,56.053812,0.175436,0.0,7.101719


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92629 entries, 0 to 92628
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Time     92629 non-null  object 
 1   BS       92629 non-null  object 
 2   Energy   92629 non-null  float64
 3   load     92629 non-null  float64
 4   ESMODE   92629 non-null  float64
 5   TXpower  92629 non-null  float64
dtypes: float64(4), object(2)
memory usage: 4.2+ MB


In [34]:
df['Time'] = pd.to_datetime(df['Time'], format='%Y%m%d %H%M%S')

In [35]:
df.describe()

,Time,Energy,load,ESMODE,TXpower
count,92629,92629.000000,92629.000000,92629.000000,92629.000000
mean,2023-01-04 00:31:21.324423424,28.138997,0.244705,0.081361,6.765427
min,2023-01-01 01:00:00,0.747384,0.000000,0.000000,5.381166
25%,2023-01-02 11:00:00,18.236173,0.057370,0.000000,6.427504
50%,2023-01-03 20:00:00,24.065770,0.165550,0.000000,6.875934
75%,2023-01-05 14:00:00,35.724963,0.363766,0.000000,6.875934
max,2023-01-08 00:00:00,100.000000,0.993957,4.000000,8.375336
std,NaN,13.934645,0.234677,0.382317,0.309929


In [36]:
df.isnull().sum()

Time       0
BS         0
Energy     0
load       0
ESMODE     0
TXpower    0
dtype: int64

In [ ]:
# Generate a profiling report
profile = pandas_profiling.ProfileReport(df)
profile.to_file("5G_Energy.html")

In [ ]:
# Identify outliers using box plots
for col in data.select_dtypes(include=[np.number]):
    plt.figure()
    sns.boxplot(data[col])
    plt.title(f"Box plot for {col}")
    plt.show()

In [38]:
df['BS'].value_counts()

BS
B_583     130
B_261     128
B_728     128
B_669     128
B_512     126
         ... 
B_1018     16
B_921      14
B_905      13
B_854       1
B_835       1
Name: count, Length: 923, dtype: int64

In [39]:
pip install category_encoders

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [40]:
import category_encoders as ce
encoder = ce.TargetEncoder(cols=['BS'])
df['BS_encoded'] = encoder.fit_transform(df['BS'], df['Energy'])

In [41]:
df = df.drop('BS', axis=1)

In [42]:
target_correlations = df.corr()['Energy'].apply(abs).sort_values()

In [43]:
target_correlations

Time          0.030404
ESMODE        0.271783
TXpower       0.450760
load          0.643022
BS_encoded    0.922452
Energy        1.000000
Name: Energy, dtype: float64

In [45]:
X = df[['ESMODE', 'TXpower', 'load', 'BS_encoded']]
y = df['Energy']

In [46]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)

In [48]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

linear_reg = LinearRegression()

linear_reg.fit(X_train_scaled, y_train)

y_pred_linear = linear_reg.predict(X_test_scaled)

rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_linear))
r2_linear = r2_score(y_test, y_pred_linear)

print(f"RMSE: {rmse_linear:.4f}\n R²: {r2_linear:.4f}\n")

RMSE: 4.1893
 R²: 0.9084

